In [0]:
import numpy as np

In [0]:
from google.colab import drive

MOUNT_PATH = "/content/gdrive"
drive.mount(MOUNT_PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

MOUNT_PATH = "/content/gdrive"

DATA_PATH = MOUNT_PATH + "/My Drive/Kaggle/data"

train_images = np.load(DATA_PATH + '/train_images.npy', encoding='latin1')
print("Train data shape : ", train_images.shape)

train_labels = np.genfromtxt(DATA_PATH + '/train_labels.csv', names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'S5')])
print("Train labels shape : ", train_labels.shape)

test_images = np.load(DATA_PATH + '/test_images.npy', encoding='latin1')
print("Test data shape : ", test_images.shape)

Train data shape :  (10000, 2)
Train labels shape :  (10000,)
Test data shape :  (10000, 2)


In [0]:
def extract_column(array, index=1):
  return np.array([array[i][index] for i in range(len(array))])

In [0]:
X = extract_column(train_images, 1)
X.shape

(10000, 10000)

In [0]:
T = extract_column(test_images, 1)
T.shape

(10000, 10000)

In [0]:
labels = extract_column(train_labels, 1)
labels.shape

(10000,)

In [0]:
category_bytes = np.unique(labels)

print("Original category labels : ",category_bytes)
    
categories = {}

for i in range(len(category_bytes)):
    categories[category_bytes[i]] = i  
    
print("\n\nIntegerized labels : ",categories.values())

Original category labels :  [b'apple' b'empty' b'moust' b'mouth' b'mug' b'nail' b'nose' b'octag'
 b'paint' b'panda' b'parro' b'peanu' b'pear' b'penci' b'pengu' b'pillo'
 b'pinea' b'pool' b'rabbi' b'rhino' b'rifle' b'rolle' b'sailb' b'scorp'
 b'screw' b'shove' b'sink' b'skate' b'skull' b'spoon' b'squig']


Integerized labels :  dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])


In [0]:
y = np.array([ categories[label] for label in labels])  

In [0]:
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

X.shape =  (10000, 10000)
y.shape =  (10000,)


In [0]:
def split_data(X,y,test_percentage = 25,validation = False):

  split_index = (100 - test_percentage) * len(X) // 100

  X_train, X_test = X[:split_index], X[split_index:]
  y_train, y_test = y[:split_index], y[split_index:]
  
  if(not validation):
    return (X_train, y_train), (X_test, y_test)
  else:
    
    split_index = (100 - test_percentage) * len(X_train) // 100
    
    X_train, X_valid = X_train[:split_index], X_train[split_index:]
    y_train, y_valid = y_train[:split_index], y_train[split_index:]
    
    return (X_train, y_train), (X_test, y_test), (X_valid, y_valid)


In [0]:
def get_datasets(X,y):
  
  X = np.take(X,np.random.permutation(X.shape[0]),axis=0,out=X)
  
  (X_train, y_train), (X_test, y_test) = split_data(X,y)

  print("X_train.shape = ",X_train.shape)
  print("y_train.shape = ",y_train.shape)

  print("X_test.shape = ",X_test.shape)
  print("y_test.shape = ",y_test.shape)
  
  return (X_train, y_train), (X_test, y_test)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [0]:
def run_linear_SVM(C=1.0):
  (X_train, y_train), (X_test, y_test) = get_datasets(X,y)
  clf = svm.SVC(kernel='linear', C = C)
  fit = clf.fit(X_train,y_train)
  prediction = clf.predict(X_test)
  accuracy = accuracy_score(y_test, prediction)

  print("Accuracy score using Linear SVM = ", accuracy)
  
  

In [0]:
def transform_images_to_monochrome(X):
  return (X<127).astype(np.int)
    
def normalize_images(X):
  return np.array([x/255.0 for x in X])

def center_images(X):
  return np.array([(x-np.mean(x))/np.std(x) for x in X])
    

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
X = transform_images_to_monochrome(X)

In [0]:
run_linear_SVM(C=1.0)

Accuracy score using Linear SVM =  0.0504


In [0]:
run_linear_SVM(C=1000.0)

Accuracy score using Linear SVM =  0.0504


In [0]:
run_linear_SVM(C=100.0)

Accuracy score using Linear SVM =  0.0504


In [0]:
run_linear_SVM(C=0.5)

Accuracy score using Linear SVM =  0.0504


In [0]:
run_linear_SVM(C=10)

Accuracy score using Linear SVM =  0.0504


In [0]:
def final_linear_SVM(C=1.0):
 
  clf = svm.SVC(kernel='linear', C = C)
  fit = clf.fit(normalize_images(X),y)
  prediction = clf.predict(normalize_images(T))

  print("DONE")
  
  return prediction
  


In [0]:
def final_random_classifier():
 
  clf = RandomForestClassifier(n_jobs=4, random_state=0)
  fit = clf.fit(normalize_images(X),y)
  prediction = clf.predict(normalize_images(T))

  print("DONE")
  
  return prediction


In [0]:
['apple' 'empty' 'moust', 'mouth', 'mug', 'nail', 'nose', 'octag', 'paint', 'panda', 'parro', 'peanu', 'pear', 'penci' b'pengu' b'pillo'
 b'pinea' b'pool' b'rabbi' b'rhino' b'rifle' b'rolle' b'sailb' b'scorp'
 b'screw' b'shove' b'sink' b'skate' b'skull' b'spoon' b'squig']

In [0]:
prediction = final_linear_SVM()

DONE


In [0]:
ys = [str(category_bytes[int(p)]).split("'")[1] for p in prediction]

In [0]:
import pandas as pd 

ys = np.array([str(category_bytes[int(p)]).split("'")[1] for p in prediction])
ids = np.array(range(10000))

sub_df = pd.DataFrame({'Id':ids,'Category':ys})
sub_df[['Id','Category']].to_csv(DATA_PATH +'/baseline_svm_submission.csv', index=False)